# SF Crime Data Preparation

- Authors: Henry Gräser, Jonas Müller, Thomas Wolff, Hannes Harnisch
- Created on: June 28, 2024
- Description: Preparing the data of the kaggle dataset SF Crime
- Kaggle competition: [SF Crime Classification](https://www.kaggle.com/c/sf-crime/data) 

## Libraries

In [113]:
import pandas as pd
# For holidays
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
# For timezone and sun position
import pytz
from astral import LocationInfo
from astral.sun import sun

import re
import os

from sklearn.preprocessing import StandardScaler

## Data import

In [114]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

train.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


## Data cleaning

### Removing Duplicates

In [115]:
train.drop_duplicates(inplace=True)
train

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541
...,...,...,...,...,...,...,...,...,...
878044,2003-01-06 00:15:00,ROBBERY,ROBBERY ON THE STREET WITH A GUN,Monday,TARAVAL,NONE,FARALLONES ST / CAPITOL AV,-122.459033,37.714056
878045,2003-01-06 00:01:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,INGLESIDE,NONE,600 Block of EDNA ST,-122.447364,37.731948
878046,2003-01-06 00:01:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,SOUTHERN,NONE,5TH ST / FOLSOM ST,-122.403390,37.780266
878047,2003-01-06 00:01:00,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Monday,SOUTHERN,NONE,TOWNSEND ST / 2ND ST,-122.390531,37.780607


### Coordinate Cleaning

Renaming Coordinate Columns and removing wrong data that is not in SF in train dataset

In [116]:
train = train.rename(columns={'X': 'long', 'Y': 'lat'})
test = test.rename(columns={'X': 'long', 'Y': 'lat'})
train = train[train['lat'] != train['lat'].max()]

### Setting Types

In [117]:
for df in [train, test]:
    df['DayOfWeek'] = df['DayOfWeek'].astype('category')
    df['PdDistrict'] = df['PdDistrict'].astype('category')

## Feature Development

### Temporal Features

We want to format the Dates collumn and also extract temporal features such as Year, Month, Day, Hour, Minute

In [118]:
train["Dates"] = pd.to_datetime(train["Dates"], format="%Y-%m-%d %H:%M:%S")
test["Dates"] = pd.to_datetime(test["Dates"], format="%Y-%m-%d %H:%M:%S")

In [119]:
def create_column(df, datetime_column, part_name):
    df[part_name.capitalize()] = df[datetime_column].map(lambda x: getattr(x, part_name, None))

for df in [train, test]:
    for part in ["year", "month", "day", "hour", "minute"]:
        create_column(df, "Dates", part)
train.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,long,lat,Year,Month,Day,Hour,Minute
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,23,53
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,23,53
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015,5,13,23,33
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873,2015,5,13,23,30
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541,2015,5,13,23,30


### Holiday Feature

Enriching data with information of Holidays:

In [120]:
cal = calendar()
for df in [train, test]:
    holidays = cal.holidays(start=df['Dates'].min(), end=df['Dates'].max())
    df['Holiday'] = (df['Dates'].dt.date.astype('datetime64[ns]').isin(holidays)).astype(int)
    print(df['Holiday'].value_counts())

Holiday
0    850060
1     25599
Name: count, dtype: int64
Holiday
0    865946
1     18316
Name: count, dtype: int64


### Night Feature

Enriching data with information on If it is at night or during the daylight:

In [121]:
def get_all_sunset_sunrise_sf(x):
    city = LocationInfo("San Francisco", "USA", "America/Los_Angeles", 37.7749, -122.4194)
    timezone = pytz.timezone(city.timezone)
    return {
        f"{day['Day']}-{day['Month']}-{day['Year']}": sun(
            city.observer,
            date=pd.Timestamp(year=day['Year'], month=day['Month'], day=day['Day'], tz=timezone).date(),
            tzinfo=city.timezone
        )
        for _, day in x.iterrows()
    }

def is_at_night(date, sun_info):
    dusk = sun_info['dusk'].replace(tzinfo=None)
    dawn = sun_info['dawn'].replace(tzinfo=None)
    # Keine Änderung hier, da die Logik korrekt ist, aber stellen Sie sicher, dass 'date' auch ohne Zeitzone ist
    if dawn < dusk:  # Für Fälle, in denen der Sonnenaufgang als am nächsten Tag betrachtet wird
        return date > dusk or date < dawn
    else:
        return dusk < date < dawn

for df in [train, test]:
    unique_days = df[['Day', 'Month', 'Year']].drop_duplicates()
    sun_info = get_all_sunset_sunrise_sf(unique_days)
    df['Night'] = df['Dates'].map(lambda x: int(is_at_night(x, sun_info[f"{x.day}-{x.month}-{x.year}"])))
    print(df['Night'].value_counts()) 

# Checking that there is no values during the middle of the day that are marked as night
train[(train['Night'] == True) & (train['Hour'] < 17) & (train['Hour'] > 7)].shape

Night
0    542681
1    332978
Name: count, dtype: int64
Night
0    547181
1    337081
Name: count, dtype: int64


(0, 16)

### Season Feature

In [122]:
seasons = { 1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Fall' }
for df in [train, test]:
    df['Season'] = df['Month'].map(lambda x: seasons[(x%12 + 3)//3])

train.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,long,lat,Year,Month,Day,Hour,Minute,Holiday,Night,Season
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,23,53,0,1,Spring
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,23,53,0,1,Spring
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015,5,13,23,33,0,1,Spring
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873,2015,5,13,23,30,0,1,Spring
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541,2015,5,13,23,30,0,1,Spring


### Address Features

In [123]:
def get_block(address):
    match = re.search(r'(\d+)\s+block of', address, re.IGNORECASE)
    if match:
        # The block number is divided by 100 because they always increase by 100 and then increased by 1 to leave 0 for no block
        return int(match.group(1)) // 100 + 1
    return 0

def get_street_type(address):
    # See also data-understanding.ipynb
    street_types = ['AV', 'ST', 'CT', 'PZ', 'LN', 'DR', 'PL', 'HY', 'FY', 'WY', 'TR', 'RD', 'BL', 'WAY', 'CR', 'AL', 'I-80', 'RW', 'WK']
    match = re.findall(r'\b(?:' + '|'.join(street_types) + r')\b', address, re.IGNORECASE)
    if len(match) > 1 and '/' in address:
        return "INT"
    if len(match) == 1:
        return match[0]
    return "OTHER"


for df in [train, test]:
    df['Block'] = df['Address'].map(get_block)
    df['StreetType'] = df['Address'].map(get_street_type)

In [124]:
train['Block'].value_counts()

Block
0     260337
1      76048
2      51811
9      51364
3      38333
       ...  
82         7
79         5
81         4
80         3
84         3
Name: count, Length: 85, dtype: int64

In [125]:
train['StreetType'].value_counts()

StreetType
ST       445602
INT      260147
AV       122090
BL        14692
DR        11902
WY         5172
RD         3604
CT         2793
PZ         2346
HY         2270
LN         1667
TR         1113
PL         1070
CR          358
I-80        322
OTHER       271
AL          179
WAY          55
WK            5
RW            1
Name: count, dtype: int64

In [126]:
for df in [train, test]:
    df['Season'] = df['Season'].astype('category')
    df['StreetType'] = df['StreetType'].astype('category')

In [127]:
# Check that an intersection has no block number
train[(train['Block'] != 0) & (train['StreetType'] == 'INT')].shape

(0, 19)

### Dropping unnecesary Dimensions

In [128]:
# Dropping the Resolution and Descript in training dataset because they should not be relevant for prediction
train = train.drop(columns=['Resolution', 'Descript'])

# Dropping the Dates column because we already extracted the relevant information
train = train.drop(columns=['Dates'])
test = test.drop(columns=['Dates'])

# Dropping the Address column because we already extracted the relevant information
train = train.drop(columns=['Address'])
test = test.drop(columns=['Address'])

# Dropping Years because it is not necessary for future predictions
train = train.drop(columns=['Year'])

In [129]:
# Sort the columns to be more organized
train = train[['DayOfWeek', 'Day', 'Month', 'Hour', 'Minute', 'Season', 'Night', 'Holiday', 'Block', 'StreetType', 'PdDistrict', 'lat', 'long', 'Category']]
test = test[['Id', 'DayOfWeek', 'Day', 'Month', 'Hour', 'Minute', 'Season', 'Night', 'Holiday', 'Block', 'StreetType', 'PdDistrict', 'lat', 'long']]

train.head()

,DayOfWeek,Day,Month,Hour,Minute,Season,Night,Holiday,Block,StreetType,PdDistrict,lat,long,Category
0,Wednesday,13,5,23,53,Spring,1,0,0,INT,NORTHERN,37.774599,-122.425892,WARRANTS
1,Wednesday,13,5,23,53,Spring,1,0,0,INT,NORTHERN,37.774599,-122.425892,OTHER OFFENSES
2,Wednesday,13,5,23,33,Spring,1,0,0,INT,NORTHERN,37.800414,-122.424363,OTHER OFFENSES
3,Wednesday,13,5,23,30,Spring,1,0,16,ST,NORTHERN,37.800873,-122.426995,LARCENY/THEFT
4,Wednesday,13,5,23,30,Spring,1,0,2,ST,PARK,37.771541,-122.438738,LARCENY/THEFT


Summarize categories with low occuences in test dataset

In [130]:
def get_unique_categories_count(df):
    return len(df["Category"].value_counts().reset_index())

In [131]:
print(get_unique_categories_count(train))
print(train["Category"].value_counts())

39
Category
LARCENY/THEFT                  174305
OTHER OFFENSES                 125943
NON-CRIMINAL                    91911
ASSAULT                         76811
DRUG/NARCOTIC                   53919
VEHICLE THEFT                   53697
VANDALISM                       44580
WARRANTS                        42137
BURGLARY                        36599
SUSPICIOUS OCC                  31392
MISSING PERSON                  25669
ROBBERY                         22987
FRAUD                           16637
FORGERY/COUNTERFEITING          10592
SECONDARY CODES                  9979
WEAPON LAWS                      8550
PROSTITUTION                     7446
TRESPASS                         7317
STOLEN PROPERTY                  4536
SEX OFFENSES FORCIBLE            4379
DISORDERLY CONDUCT               4311
DRUNKENNESS                      4277
RECOVERED VEHICLE                3132
KIDNAPPING                       2340
DRIVING UNDER THE INFLUENCE      2268
LIQUOR LAWS                      1899


Summarizing crime categories with count below 1000 in train dataset.

In [132]:
# Train
train['Category'] = train['Category'].replace('TREA', 'OTHER OFFENSES')
train['Category'] = train['Category'].replace('PORNOGRAPHY/OBSCENE MAT', 'OTHER OFFENSES')
train['Category'] = train['Category'].replace('GAMBLING', 'OTHER OFFENSES')
train['Category'] = train['Category'].replace('SEX OFFENSES NON FORCIBLE', 'OTHER OFFENSES')
train['Category'] = train['Category'].replace('EXTORTION', 'OTHER OFFENSES')
train['Category'] = train['Category'].replace('BRIBERY', 'OTHER OFFENSES')
train['Category'] = train['Category'].replace('BAD CHECKS', 'OTHER OFFENSES')
train['Category'] = train['Category'].replace('FAMILY OFFENSES', 'OTHER OFFENSES')
train['Category'] = train['Category'].replace('SUICIDE', 'OTHER OFFENSES')

In [133]:
get_unique_categories_count(train)

30

### Export prepared data

In [134]:
tmp_dir = 'data/tmp'

# Check if the directory exists, and create it if it doesn't
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)
    
train.to_csv('data/tmp/prepared_train.csv', index=True)
test.to_csv('data/tmp/prepared_test.csv', index=True)

## Data encoding

In [135]:
encoded_train = train.copy() 
encoded_test = test.copy()

In [136]:
# Transforming Categorical attributes -> Nummerical Attributes according to Slide 25 Data Perparation
def create_columns_for_unique_values(df, column):
    unique_values = df[column].unique()
    for value in unique_values:
        df[column + "-" + value] = (df[column] == value).astype(int)

for df in [encoded_train, encoded_test]:
    columns = ['DayOfWeek', 'PdDistrict', 'StreetType', 'Season']
    for column in columns:
        create_columns_for_unique_values(df, column)
    df.drop(columns=columns, inplace=True)

encoded_train.head()

,Day,Month,Hour,Minute,Night,Holiday,Block,lat,long,Category,...,StreetType-CR,StreetType-OTHER,StreetType-AL,StreetType-WK,StreetType-RW,StreetType-I-80,Season-Spring,Season-Winter,Season-Fall,Season-Summer
0,13,5,23,53,1,0,0,37.774599,-122.425892,WARRANTS,...,0,0,0,0,0,0,1,0,0,0
1,13,5,23,53,1,0,0,37.774599,-122.425892,OTHER OFFENSES,...,0,0,0,0,0,0,1,0,0,0
2,13,5,23,33,1,0,0,37.800414,-122.424363,OTHER OFFENSES,...,0,0,0,0,0,0,1,0,0,0
3,13,5,23,30,1,0,16,37.800873,-122.426995,LARCENY/THEFT,...,0,0,0,0,0,0,1,0,0,0
4,13,5,23,30,1,0,2,37.771541,-122.438738,LARCENY/THEFT,...,0,0,0,0,0,0,1,0,0,0


In [137]:
scaler = StandardScaler()
encoded_train[['lat', 'long']] = scaler.fit_transform(encoded_train[['lat', 'long']])
encoded_test[['lat', 'long']] = scaler.transform(encoded_test[['lat', 'long']])

encoded_train.head()

,Day,Month,Hour,Minute,Night,Holiday,Block,lat,long,Category,...,StreetType-CR,StreetType-OTHER,StreetType-AL,StreetType-WK,StreetType-RW,StreetType-I-80,Season-Spring,Season-Winter,Season-Fall,Season-Summer
0,13,5,23,53,1,0,0,0.312945,-0.123472,WARRANTS,...,0,0,0,0,0,0,1,0,0,0
1,13,5,23,53,1,0,0,0.312945,-0.123472,OTHER OFFENSES,...,0,0,0,0,0,0,1,0,0,0
2,13,5,23,33,1,0,0,1.381231,-0.063006,OTHER OFFENSES,...,0,0,0,0,0,0,1,0,0,0
3,13,5,23,30,1,0,16,1.400196,-0.167127,LARCENY/THEFT,...,0,0,0,0,0,0,1,0,0,0
4,13,5,23,30,1,0,2,0.186425,-0.631594,LARCENY/THEFT,...,0,0,0,0,0,0,1,0,0,0


### Save encoded data

In [138]:
# Save processed data
encoded_train.to_csv("data/tmp/encoded_train.csv", index=False)
encoded_test.to_csv("data/tmp/encoded_test.csv", index=False)